In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
import os
os.chdir('..')

In [2]:
from textlong import create_reason_agent, ChatZhipuAI
from langchain_openai import ChatOpenAI
from langchain_experimental.tools import PythonREPLTool

tools = [PythonREPLTool()]
llm_zhipu = ChatZhipuAI(model="glm-4")
llm_openai = ChatOpenAI(model="gpt-4-0125-preview")

agent1 = create_reason_agent(llm_zhipu, tools=tools)
agent2 = create_reason_agent(llm_openai, tools=tools)

In [3]:
question1 = """
    我不知道该怎么设置6位数字密码，请你帮我编写一个计算肥波那契数列的函数，并在前20个数字中随机选择运数字，合成一个6位数的数字串作为我的密码。
    """
question2 = "写一个关于程序员的糗事小作文"


In [4]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent2, tools=tools, handle_parsing_errors=True)

async for chunk in agent_executor.astream_events({"input": question2}, version="v1"):
    event = chunk['event']
    if(event == "on_chat_model_stream"):
        if('chunk' in chunk['data']):
            print(chunk['data']['chunk'].content, end="_", flush=True)

/Users/xuehongwei/Library/Caches/pypoetry/virtualenvs/langchain-chinese-gSQlHcwW-py3.9/lib/python3.9/site-packages/langchain_core/_api/beta_decorator.py:86: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


1_._ 关_键_概_念_:_ 程_序_员_的_糗_事_小_作_文_
_  _ -_ 没_有_明_确_给_出_具_体_取_值_。

_2_._ 概_念_拆_解_:
_  _ -_ 程_序_员_的_糗_事_
_    _ -_ 程_序_错误_导_致_的_糗_事_
_    _ -_ 工_作_中_的_沟_通_失_误_
_    _ -_ 对_技_术_误_解_或_错误_应_用_

_3_._ 反_思_:
_  _ -_ 考_虑_到_程序_员_的_糗_事_可能_与_技_术_操作_、_沟_通_以_及_对_技_术_的_理_解_有_关_，_需要_结_合_常_见_的_场_景_来_构_建_故_事_。
_  _ -_ 反_思_中_没有_涉_及_具_体_场_景_的_示_例_，_可以_考_虑_增_加_一_些_典_型_例_子_以_便_构_建_故_事_。
_  _ -_ 已_经_明_确_了_糗_事_的_几_个_可能_来源_，_但_还_没有_确定_最_终_选择_哪_一个_场_景_。

_4_._ 思_考_:
_  _ -_ 确_定_一个_具_体_的_场_景_更_有_助_于_写_出_吸_引_人_的_故_事_。
_  _ -_ 考_虑_到_程序_员_的_工_作_特_性_，_一个_关_于_代码_错误_导_致_意_想_不_到_后_果_的_场_景_可能_既_好_理_解_又_引_人_入_胜_。
_  _ -_ _选择_一个_既_能_反_映_程序_员_日_常_工_作_，_又_能_带_来_幽_默_感_的_场_景_作_为_故_事_背_景_。

_5_._ 推_理_:
_  _ -_ 根_据_程序_员_工_作_的_性_质_，_代码_错误_是_比_较_常_见_的_糗_事_来源_之_一_。_一个_简_单_的_逻_辑_错误_或_拼_写_错误_可能_导_致_不_预_期_的_结果_，_这_可以_作_为_故_事_的_基_础_。
_  _ -_ _选择_一个_具_体_的_编_程_错误_作_为_故_事_的_开_端_，_然_后_详_述_其_带_来_的_连_锁_反_应_，_以_及_最_后_的_解_决_过_程_。

_6_._ 计_划_:
_  _ -_ 动_作_:_ _无_需_使用_工_具_，_直_接_构_建_小_故_事_。
_  _ -_ 故_事_背_景_:_ 程_序_员_在_晚_上_加_班_疲_惫_之_际_，_处理_一个_看_似_

In [8]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent2, tools=tools, handle_parsing_errors=True)

async for chunk in agent_executor.astream_events({"input": question1}, version="v1"):
    event = chunk['event']
    if(event == "on_chat_model_stream"):
        if('chunk' in chunk['data']):
            print(chunk['data']['chunk'].content, end="_", flush=True)

任务_完成_后_，_我_生成_了_一个_斐_波_那_契_数_列_的_函数_，并_从_前_20_个_数字_中_随_机_选择_了_6_个_数字_组_成_了_一个_密码_串_。_但_是_，_我_注意_到_生成_的_数字_串_长度_超_过_了_6_位_。_这_是_因_为_我_直_接_从_斐_波_那_契_数_列_中_随_机_选择_了_数字_，_而_没有_考_虑_到_这_些_数字_的_长度_。_我_需要_重新_规_划_以_确_保_生成_的_密码_串_恰_好_为_6_位_数_。_计_划_是_选择_更_小_的_斐_波_那_契_数_或_者_采_取_不_同_的_选择_策_略_来_确_保_最_终_的_密码_长度_为_6_位_。

_为_了_确_保_生成_的_密码_长度_为_6_位_，_我_将_重新_调_整_策_略_，_优_先_选择_斐_波_那_契_序_列_中_较_小_的_数字_，并_可能_需要_更_加_精_细_地_控_制_数字_的_选择_过_程_，_以_确_保_最_终_拼_接_的_密码_长度_恰_好_为_6_位_数_。__我_生成_了_一个_6_位_数_的_密码_串_：_987_255_。_这_个_密码_是_通过_从_斐_波_那_契_数_列_的_前_20_个_数字_中_随_机_选择_组_成_的_，_确_保_了_密码_长度_恰_好_为_6_位_。__我_生成_了_一个_6_位_数_的_密码_：_322_831_。_这_个_密码_是_通过_从_斐_波_那_契_数_列_的_前_20_个_数字_中_随_机_选择_并_适_当_调_整_以_确_保_长度_恰_好_为_6_位_数_。_这_样_的_密码_结_合_了_随_机_性_和_一_定_的_数_学_美_感_，_同时_也_考_虑_到_了_安_全_性_的_基_本_要_求_。

_```_json_
_{"_name_":_ "_FIN_ISH_",_ "_args_":_ {"_output_":_ "_322_831_"}}
_```__

In [7]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent1, tools=tools, handle_parsing_errors=True)

async for chunk in agent_executor.astream_events({"input": question1}, version="v1"):
    event = chunk['event']
    if(event == "on_chat_model_stream"):
        if('chunk' in chunk['data']):
            print(chunk['data']['chunk'].content, end="_", flush=True)

关键_概念_:
-_ 肥_波_那_契_数_列_
-_ _6_位数_密码_
-_ 随_机_选择_

概念_拆_解_:
-_ 肥_波_那_契_数_列_ -_ 
 _ -_ -_ 定义_
 _ -_ -_ 前_20_个_数字_
-_ _6_位数_密码_ -_ 
 _ -_ -_ 长_度_
 _ -_ -_ 组_成_数字_来源_
-_ 随_机_选择_ -_ 
 _ -_ -_ 方_式_
 _ -_ -_ 数_量_

反思_:
-_ 我_需要_先_了解_肥_波_那_契_数_列_的定义_，_才能_正确_生成_它_。
-_ 我_需要_确定_如何_从_肥_波_那_契_数_列_的前_20_个_数字_中_随机_选择_数字_。
-_ 我_要_确保_生成的_6_位数_密码_中的_每个_数字_都_来自于_肥_波_那_契_数_列_。

思考_:
-_ 我_将_使用_Python__RE_PL_来_编写_生成_肥_波_那_契_数_列_的_函数_，_并_从中_随机_选择_数字_。
-_ 我_需要_确保_生成的_密码_是_6_位_数字_。

推理_:
-_ 我_将_编写_一个_函数_计算_肥_波_那_契_数_列_的前_20_个_数字_。
-_ 使用_Python_的_random_库_来_随机_选择_6_个_数字_。
-_ 将_这_6_个_随机_选择的_数字_组合_成一个_6_位数_作为_密码_。

计划_:
-_ 使用_Python__RE_PL_编写_一个_计算_肥_波_那_契_数_列_的_函数_，_并_从中_随机_选择_6_个_数字_生成_密码_。

```_json_
{"_name_":_ "_Python__RE_PL_",_ "_args_":_ {"_query_":_ "_def_ fibonacci_(n_):\_n_   _ fib_ =_ [_0_,_ _1_]\_n_   _ while_ len_(f_ib_)_ <_ n_:\_n_       _ fib_.append_(f_ib_[-_1_]_ +_ fib_[-_2_])_\_n_   _ return_ fib_\n_\n_password__digits_ =_ random_.sample_(f_ib_onacci_(_20_),_ _6_)\_n_password_ =_ ''._join_(map_(s